### Задание по программированию: Рекомендательные системы

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

### Задание

    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
    Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

    Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
    Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_table('coursera_sessions_train.txt', sep=";" , header = None)
test = pd.read_table('coursera_sessions_test.txt', sep=";" , header = None) 

In [8]:
train.dropna(inplace=True)

In [41]:
from collections import Counter
def counter_of(series):
    dct = Counter(series.split(',')).most_common()
    return dct    

In [42]:
train['checked'] = train[0].apply(counter_of)

In [43]:
train

,0,1,checked
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[(60, 2), (61, 2), (67, 2), (59, 1), (62, 1), ..."
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[(84, 2), (86, 2), (85, 1), (87, 1), (88, 1), ..."
19,"138,198,199,127",199,"[(138, 1), (198, 1), (199, 1), (127, 1)]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[(303, 1), (304, 1), (305, 1), (306, 1), (307,..."
33,"352,353,352",352,"[(352, 2), (353, 1)]"
55,519,519,"[(519, 1)]"
64,"599,600,601,602","603,604,602,599,605,606,600","[(599, 1), (600, 1), (601, 1), (602, 1)]"
72,"687,688,689,690,691,690,688,690,688,692","690,688","[(688, 3), (690, 3), (687, 1), (689, 1), (691,..."
89,"850,851,852",851,"[(850, 1), (851, 1), (852, 1)]"
93,"879,884,170,137,170,879,884,879,885,886,879,88...",879,"[(879, 9), (884, 4), (343, 4), (170, 3), (886,..."
